<a href="https://colab.research.google.com/github/KevinGJ23/Informatica_medica_2025/blob/main/modelos_pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importacion**

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.3 MB/s eta 0:00:00


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00


##**Librerias**

In [ ]:
#Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from category_encoders import CatBoostEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import optuna
pd.set_option('display.max_columns', None)

##**Funciones**

In [ ]:
#Funcion resumen de datos
def resumen(dataframe):
  # Cálculos de resumen
  cantidad_nulos= dataframe.isnull().sum()
  cantidad_total = len(dataframe)
  cantidad_no_nulos = dataframe.notnull().sum()
  tipo_de_datos = dataframe.dtypes
  cantidad_de_clases = dataframe.nunique()
  # Crear DataFrame de resumen
  resumen_df = pd.DataFrame({
      'Tipo de Dato': tipo_de_datos,
      'Valores Únicos': cantidad_de_clases,
      'No Nulos': cantidad_no_nulos,
      'Nulos': cantidad_nulos,
      '% Nulos': (cantidad_nulos / cantidad_total) * 100
  })
  # Ordenar por % de nulos
  resumen_df = resumen_df.sort_values(by='% Nulos', ascending=False)
  # Aplicar formato con Styler
  tabla_resumen = resumen_df.style \
      .format({'% Nulos': '{:.2f}%'}) \
      .bar(subset='% Nulos', color='#f08080') \
      .set_properties(**{'text-align': 'center'}) \
      .set_caption("Resumen de Columnas del DataFrame") \
      .set_table_styles([
          # Color del encabezado de columnas
          {'selector': 'th', 'props': [
              ('text-align', 'center'),
              ('background-color', '#1f4e79'),
              ('color', 'white'),
              ('font-weight', 'bold')
          ]},
          # Color de las celdas
          {'selector': 'td', 'props': [('padding', '5px')]},
          # Estilo del título de la tabla
          {'selector': 'caption', 'props': [
              ('color', '#1f4e79'),
              ('font-size', '18px'),
              ('text-align', 'center'),
              ('font-weight', 'bold')
          ]}
      ])
  # Mostrar
  display(tabla_resumen)
  # Información final
  print(f"\n🧾 El DataFrame tiene un total de **{len(dataframe.columns)}** columnas y **{len(dataframe)}** filas.")

In [ ]:
#Función Encabezado
def encabezado(dataframe,filas=5):
  salida=dataframe.head(filas).style \
      .set_properties(**{
          'background-color': '#f9f9f9',
          'border': '1px solid #ddd',
          'text-align': 'center'
      }) \
      .set_table_styles([
          {'selector': 'th', 'props': [('background-color', '#4B8BBE'), ('color', 'white'), ('text-align', 'center')]}
      ]) \
      .highlight_null() \
      .set_caption("🔍 Vista Previa de los Primeros Registros")
  display(salida)

In [ ]:
X=pd.read_csv('X.csv')
y=pd.read_csv('y.csv')

##**Visualizacion**

In [ ]:
y=y.drop(columns=['GRD_AGRUPADO_2'])

In [ ]:
encabezado(X)

,SEXO,TIPO_INGRESO,CUIDADOS_INTENSIVOS,DXR_1,DXR_2,DXR_3,DXR_4,DXR_5,DXR_6,DXR_7,DXR_8,DXR_9,SITUACION_ALTA,TIPO_SERVICIO,INFECCIONES,INFECCION_QUIRURGICA,DX_EGRESO_AGRUPADO,PROC_1_GRUPO,PROC_2_GRUPO,PROC_3_GRUPO,PROC_4_GRUPO,PROC_5_GRUPO,PROC_6_GRUPO,PROC_7_GRUPO,PROC_8_GRUPO,PROC_9_GRUPO,PROC_10_GRUPO,PROC_11_GRUPO,PROC_12_GRUPO,PROC_13_GRUPO,MES,DIA_SEMANA,EDAD_AGRUPADA,ESTANCIA_AGRUPADA,UCI_AGRUPADA
0,F,URGENCIA,NO,Circulatorio,Neoplasias,Neoplasias,Neoplasias,Genitourinario,Infecciosas,Neoplasias,Otros,Vacio,ALTA MÉDICA,URGENCIA ADULTOS,NO,NO,Circulatorio,Genitourinario,Genitourinario,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,5,Lunes,Joven adulto,15+ días,Sin cuidados
1,M,URGENCIA,NO,Infecciosas,Infecciosas,Infecciosas,Infecciosas,Neoplasias,Infecciosas,Endocrino,Neoplasias,Infecciosas,ALTA MÉDICA,NO APLICA,NO,NO,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,3,Viernes,Joven adulto,15+ días,Sin cuidados
2,M,URGENCIA,NO,Infecciosas,Neurológico,Infecciosas,Neoplasias,Neoplasias,Infecciosas,Neoplasias,Endocrino,Endocrino,ALTA MÉDICA,URGENCIA ADULTOS,NO,NO,Infecciosas,Digestivo,Genitourinario,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,3,Domingo,Joven adulto,15+ días,Sin cuidados
3,M,URGENCIA,NO,Infecciosas,Infecciosas,Infecciosas,Genitourinario,Infecciosas,Infecciosas,Neoplasias,Otros,Vacio,ALTA MÉDICA,URGENCIA ADULTOS,NO,NO,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,6,Lunes,Joven adulto,2-4 días,Sin cuidados
4,M,URGENCIA,NO,Traumatismo,Neoplasias,Genitourinario,Infecciosas,Infecciosas,Infecciosas,Neoplasias,Endocrino,Otros,ALTA MÉDICA,URGENCIA ADULTOS,NO,NO,Traumatismo,Genitourinario,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,11,Miércoles,Adulto,15+ días,Sin cuidados


In [ ]:
encabezado(y)

,GRD_ORIGINAL,CODIGOS_SOLOS,GRD_GRUPO_2DIG,GRD_AGRUPADO
0,14143,OTROS,14,14
1,184153,OTROS,184,184
2,184153,OTROS,184,184
3,184153,OTROS,184,184
4,184112,OTROS,184,184


In [ ]:
resumen(y)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
GRD_ORIGINAL,int64,608,27869,0,0.00%
CODIGOS_SOLOS,object,107,27869,0,0.00%
GRD_GRUPO_2DIG,int64,34,27869,0,0.00%
GRD_AGRUPADO,int64,107,27869,0,0.00%



🧾 El DataFrame tiene un total de **4** columnas y **27869** filas.


In [ ]:
resumen(X)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SEXO,object,2,27869,0,0.00%
TIPO_INGRESO,object,2,27869,0,0.00%
CUIDADOS_INTENSIVOS,object,2,27869,0,0.00%
DXR_1,object,15,27869,0,0.00%
DXR_2,object,15,27869,0,0.00%
DXR_3,object,15,27869,0,0.00%
DXR_4,object,15,27869,0,0.00%
DXR_5,object,15,27869,0,0.00%
DXR_6,object,15,27869,0,0.00%
DXR_7,object,15,27869,0,0.00%



🧾 El DataFrame tiene un total de **35** columnas y **27869** filas.


##**Codificación**

###**Posibles Variables Redundantes**

In [ ]:
for col in X.columns:
    top_val = X[col].value_counts(normalize=True).iloc[0]
    n_unique = X[col].nunique()
    print(f"{col}: {n_unique} categorías únicas, más común = {top_val:.2%}")



SEXO: 2 categorías únicas, más común = 51.51%
TIPO_INGRESO: 2 categorías únicas, más común = 91.77%
CUIDADOS_INTENSIVOS: 2 categorías únicas, más común = 95.75%
DXR_1: 15 categorías únicas, más común = 20.62%
DXR_2: 15 categorías únicas, más común = 23.67%
DXR_3: 15 categorías únicas, más común = 22.75%
DXR_4: 15 categorías únicas, más común = 35.47%
DXR_5: 15 categorías únicas, más común = 48.15%
DXR_6: 15 categorías únicas, más común = 59.94%
DXR_7: 15 categorías únicas, más común = 70.72%
DXR_8: 15 categorías únicas, más común = 79.87%
DXR_9: 15 categorías únicas, más común = 87.67%
SITUACION_ALTA: 2 categorías únicas, más común = 94.64%
TIPO_SERVICIO: 4 categorías únicas, más común = 89.13%
INFECCIONES: 2 categorías únicas, más común = 98.16%
INFECCION_QUIRURGICA: 2 categorías únicas, más común = 99.26%
DX_EGRESO_AGRUPADO: 14 categorías únicas, más común = 17.23%
PROC_1_GRUPO: 17 categorías únicas, más común = 31.24%
PROC_2_GRUPO: 16 categorías únicas, más común = 50.74%
PROC_3_GRU

###**Importancia Variables**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y['GRD_AGRUPADO'], stratify=y['GRD_AGRUPADO'], test_size=0.2, random_state=42)

In [ ]:
from catboost import CatBoostClassifier, Pool
import pandas as pd

# Columnas categóricas (asegúrate de que sean tipo 'object' o 'category')
cat_features = X.select_dtypes(include='object').columns.tolist()

# Crear dataset para CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features)

# Crear y entrenar el modelo usando GPU
model = CatBoostClassifier(
    task_type='GPU',     # ← Activar uso de GPU
    devices='0',         # ← Selecciona la GPU (si tienes más de una, puedes cambiar)
    iterations=100,      # ← Ajusta si quieres entrenar más rápido o más preciso
    depth=6,
    learning_rate=0.1,
    verbose=50           # ← Te da progreso durante el entrenamiento
)

model.fit(train_pool)

# Obtener importancia de características
importancias = model.get_feature_importance(train_pool)
df_importancia = pd.DataFrame({
    'Feature': X.columns,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=False)

print(df_importancia)



0:	learn: 4.2315349	total: 275ms	remaining: 27.2s
50:	learn: 2.2241730	total: 12s	remaining: 11.5s
99:	learn: 1.9229900	total: 24.8s	remaining: 0us
                 Feature  Importancia
16    DX_EGRESO_AGRUPADO    69.411445
17          PROC_1_GRUPO    11.780288
3                  DXR_1     4.456092
32         EDAD_AGRUPADA     3.278266
7                  DXR_5     2.703454
1           TIPO_INGRESO     1.870580
18          PROC_2_GRUPO     1.421749
33     ESTANCIA_AGRUPADA     1.269024
30                   MES     1.111222
0                   SEXO     1.044842
2    CUIDADOS_INTENSIVOS     0.643359
19          PROC_3_GRUPO     0.532918
12        SITUACION_ALTA     0.275991
21          PROC_5_GRUPO     0.073228
34          UCI_AGRUPADA     0.052663
15  INFECCION_QUIRURGICA     0.042639
14           INFECCIONES     0.028973
20          PROC_4_GRUPO     0.003269
10                 DXR_8     0.000000
11                 DXR_9     0.000000
13         TIPO_SERVICIO     0.000000
5               

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y['GRD_GRUPO_2DIG'], stratify=y['GRD_GRUPO_2DIG'], test_size=0.2, random_state=42)

In [ ]:
from catboost import CatBoostClassifier, Pool
import pandas as pd

# Columnas categóricas (asegúrate de que sean tipo 'object' o 'category')
cat_features = X.select_dtypes(include='object').columns.tolist()

# Crear dataset para CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features)

# Crear y entrenar el modelo usando GPU
model = CatBoostClassifier(
    task_type='GPU',     # ← Activar uso de GPU
    devices='0',         # ← Selecciona la GPU (si tienes más de una, puedes cambiar)
    iterations=100,      # ← Ajusta si quieres entrenar más rápido o más preciso
    depth=6,
    learning_rate=0.1,
    verbose=50           # ← Te da progreso durante el entrenamiento
)

model.fit(train_pool)

# Obtener importancia de características
importancias = model.get_feature_importance(train_pool)
df_importancia = pd.DataFrame({
    'Feature': X.columns,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=False)

print(df_importancia)

0:	learn: 2.8136353	total: 51.8ms	remaining: 5.13s
50:	learn: 1.0830904	total: 1.76s	remaining: 1.69s
99:	learn: 0.9085978	total: 3.35s	remaining: 0us
                 Feature  Importancia
16    DX_EGRESO_AGRUPADO    60.643455
17          PROC_1_GRUPO    18.329696
3                  DXR_1     3.951529
18          PROC_2_GRUPO     3.128754
1           TIPO_INGRESO     2.380222
30                   MES     2.350511
33     ESTANCIA_AGRUPADA     2.026086
4                  DXR_2     1.469577
19          PROC_3_GRUPO     1.272268
0                   SEXO     1.202843
32         EDAD_AGRUPADA     1.058434
2    CUIDADOS_INTENSIVOS     0.764684
20          PROC_4_GRUPO     0.656736
12        SITUACION_ALTA     0.341894
23          PROC_7_GRUPO     0.159921
21          PROC_5_GRUPO     0.078811
34          UCI_AGRUPADA     0.077494
15  INFECCION_QUIRURGICA     0.060935
14           INFECCIONES     0.023999
13         TIPO_SERVICIO     0.016412
24          PROC_8_GRUPO     0.005740
5            

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y['CODIGOS_SOLOS'], stratify=y['CODIGOS_SOLOS'], test_size=0.2, random_state=42)

In [ ]:
from catboost import CatBoostClassifier, Pool
import pandas as pd

# Columnas categóricas (asegúrate de que sean tipo 'object' o 'category')
cat_features = X.select_dtypes(include='object').columns.tolist()

# Crear dataset para CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features)

# Crear y entrenar el modelo usando GPU
model = CatBoostClassifier(
    task_type='GPU',     # ← Activar uso de GPU
    devices='0',         # ← Selecciona la GPU (si tienes más de una, puedes cambiar)
    iterations=100,      # ← Ajusta si quieres entrenar más rápido o más preciso
    depth=6,
    learning_rate=0.1,
    verbose=50           # ← Te da progreso durante el entrenamiento
)

model.fit(train_pool)

# Obtener importancia de características
importancias = model.get_feature_importance(train_pool)
df_importancia = pd.DataFrame({
    'Feature': X.columns,
    'Importancia': importancias
}).sort_values(by='Importancia', ascending=False)

print(df_importancia)

0:	learn: 3.6596539	total: 390ms	remaining: 38.6s
50:	learn: 1.9130148	total: 12.5s	remaining: 12s
99:	learn: 1.6301989	total: 25.7s	remaining: 0us
                 Feature  Importancia
12        SITUACION_ALTA    31.561080
2    CUIDADOS_INTENSIVOS    16.156499
16    DX_EGRESO_AGRUPADO    16.064671
1           TIPO_INGRESO    12.199920
14           INFECCIONES    10.591048
17          PROC_1_GRUPO     3.365956
0                   SEXO     2.540165
30                   MES     1.947365
3                  DXR_1     1.292790
32         EDAD_AGRUPADA     1.250008
7                  DXR_5     1.076574
33     ESTANCIA_AGRUPADA     0.673681
4                  DXR_2     0.671108
19          PROC_3_GRUPO     0.415415
20          PROC_4_GRUPO     0.149433
15  INFECCION_QUIRURGICA     0.022799
34          UCI_AGRUPADA     0.013080
13         TIPO_SERVICIO     0.008407
10                 DXR_8     0.000000
18          PROC_2_GRUPO     0.000000
11                 DXR_9     0.000000
9               

**Analisis:**

Con base en lo anterior, se decidió conservar únicamente las primeras 14 columnas. Esta selección se hizo, en primer lugar, por limitaciones de espacio en la página web, y en segundo lugar, porque dichas variables demostraron tener la mayor relevancia en la predicción de cualquiera de los targets considerados.

In [ ]:
X=X.drop(columns=['PROC_12_GRUPO','PROC_13_GRUPO','PROC_11_GRUPO','PROC_10_GRUPO','PROC_9_GRUPO','PROC_8_GRUPO',
                        'PROC_7_GRUPO','PROC_6_GRUPO','PROC_5_GRUPO','PROC_4_GRUPO','MES','DIA_SEMANA','INFECCIONES',
                        'INFECCION_QUIRURGICA','DXR_9','DXR_8','DXR_7','DXR_6','DXR_5','DXR_4','DXR_2'])

In [ ]:
encabezado(X)

,SEXO,TIPO_INGRESO,CUIDADOS_INTENSIVOS,DXR_1,DXR_3,SITUACION_ALTA,TIPO_SERVICIO,DX_EGRESO_AGRUPADO,PROC_1_GRUPO,PROC_2_GRUPO,PROC_3_GRUPO,EDAD_AGRUPADA,ESTANCIA_AGRUPADA,UCI_AGRUPADA
0,F,URGENCIA,NO,Circulatorio,Neoplasias,ALTA MÉDICA,URGENCIA ADULTOS,Circulatorio,Genitourinario,Genitourinario,Diagnóstico/terapéutico,Joven adulto,15+ días,Sin cuidados
1,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,NO APLICA,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Joven adulto,15+ días,Sin cuidados
2,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Digestivo,Genitourinario,Diagnóstico/terapéutico,Joven adulto,15+ días,Sin cuidados
3,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Joven adulto,2-4 días,Sin cuidados
4,M,URGENCIA,NO,Traumatismo,Genitourinario,ALTA MÉDICA,URGENCIA ADULTOS,Traumatismo,Genitourinario,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Adulto,15+ días,Sin cuidados


In [ ]:
resumen(X)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SEXO,object,2,27869,0,0.00%
TIPO_INGRESO,object,2,27869,0,0.00%
CUIDADOS_INTENSIVOS,object,2,27869,0,0.00%
DXR_1,object,15,27869,0,0.00%
DXR_3,object,15,27869,0,0.00%
SITUACION_ALTA,object,2,27869,0,0.00%
TIPO_SERVICIO,object,4,27869,0,0.00%
DX_EGRESO_AGRUPADO,object,14,27869,0,0.00%
PROC_1_GRUPO,object,17,27869,0,0.00%
PROC_2_GRUPO,object,16,27869,0,0.00%



🧾 El DataFrame tiene un total de **14** columnas y **27869** filas.


##**Codificación De Features**

In [ ]:
ordinal_diagnosticos={
    'Vacio':0,
    'Infecciosas': 1,
    'Neoplasias': 2,
    'Endocrino': 3,
    'Psicológico': 4,
    'Neurológico': 5,
    'Sensorial': 6,
    'Circulatorio': 7,
    'Respiratorio': 8,
    'Digestivo': 9,
    'Dermatológico': 10,
    'Musculoesquelético': 11,
    'Genitourinario': 12,
    'Traumatismo': 13,
    'Otros':14
}
ordinal_procesos={
    'Desconocido':0,
    'Misceláneos': 1,
    'Sistema nervioso': 2,
    'Endócrino': 3,
    'Ojo y oído': 4,
    'Respiratorio': 5,
    'Cardiovascular': 6,
    'Linfático y bazo': 7,
    'Digestivo': 8,
    'Genitourinario': 9,
    'Parto y puerperio': 10,
    'Musculoesquelético': 11,
    'Tegumentario': 12,
    'Obstetricia': 13,
    'Cirugía músculo-esquelética': 14,
    'Mama y piel': 15,
    'Diagnóstico/terapéutico': 16,
    'Otros':17
}

ordinal_edad={
    'Neonato':0,
    'Niñez':1,
    'Adolescencia':2,
    'Joven adulto':3,
    'Adulto':4,
    'Adulto mayor':5
}

ordinal_dia={
    'Lunes':0,
    'Martes':1,
    'Miércoles':2,
    'Jueves':3,
    'Viernes':4,
    'Sábado':5,
    'Domingo':6
}
ordinal_dias_estancia={
    '<2 días':0,
    '2-4 días':1,
    '5-7 días':2,
    '8-14 días':3,
    '15+ días':4
}
ordinal_uci={
    'Sin cuidados':0,
    '1 día':1,
    '2-4 días':2,
    '5-9 días':3,
    '10+ días':4
}
ordinal_tipo_servicio={
    'CIRUGÍA':0,
    'URGENCIA PEDÍATRICAS':1,
    'NO APLICA':2,
    'URGENCIA ADULTOS':3
}


In [ ]:
X_nuevo=X.copy()
# Diagnósticos (puede haber varios)
for col in ['DXR_1', 'DXR_3','DX_EGRESO_AGRUPADO']:
    if col in X_nuevo.columns:
        X_nuevo[col] = X_nuevo[col].map(ordinal_diagnosticos)

# Procesos
for col in ['PROC_1_GRUPO', 'PROC_2_GRUPO', 'PROC_3_GRUPO']:
    if col in X_nuevo.columns:
        X_nuevo[col] = X_nuevo[col].map(ordinal_procesos)

# Edad
if 'EDAD_AGRUPADA' in X_nuevo.columns:
    X_nuevo['EDAD_AGRUPADA'] = X_nuevo['EDAD_AGRUPADA'].map(ordinal_edad)

# Estancia
if 'ESTANCIA_AGRUPADA' in X_nuevo.columns:
    X_nuevo['ESTANCIA_AGRUPADA'] = X_nuevo['ESTANCIA_AGRUPADA'].map(ordinal_dias_estancia)

# UCI
if 'UCI_AGRUPADA' in X_nuevo.columns:
    X_nuevo['UCI_AGRUPADA'] = X_nuevo['UCI_AGRUPADA'].map(ordinal_uci)

# Tipo de servicio
if 'TIPO_SERVICIO' in X_nuevo.columns:
    X_nuevo['TIPO_SERVICIO'] = X_nuevo['TIPO_SERVICIO'].map(ordinal_tipo_servicio)

# Mapear manualmente a 0 y 1 (ajusta según tus valores reales)

X_nuevo['SEXO'] = X_nuevo['SEXO'].map({'F': 0, 'M': 1})
X_nuevo['CUIDADOS_INTENSIVOS'] = X_nuevo['CUIDADOS_INTENSIVOS'].map({'NO': 0, 'SI': 1})
X_nuevo['TIPO_INGRESO'] = X_nuevo['TIPO_INGRESO'].map({'PROGRAMADO': 0, 'URGENCIA': 1})
X_nuevo['SITUACION_ALTA'] = X_nuevo['SITUACION_ALTA'].map({'ALTA MÉDICA': 0, 'FALLECIDO': 1})



In [ ]:
encabezado(X_nuevo)

,SEXO,TIPO_INGRESO,CUIDADOS_INTENSIVOS,DXR_1,DXR_3,SITUACION_ALTA,TIPO_SERVICIO,DX_EGRESO_AGRUPADO,PROC_1_GRUPO,PROC_2_GRUPO,PROC_3_GRUPO,EDAD_AGRUPADA,ESTANCIA_AGRUPADA,UCI_AGRUPADA
0,0,1,0,7,2,0,3,7,9,9,16,3,4,0
1,1,1,0,1,1,0,2,1,16,0,0,3,4,0
2,1,1,0,1,1,0,3,1,8,9,16,3,4,0
3,1,1,0,1,1,0,3,1,16,0,0,3,1,0
4,1,1,0,13,12,0,3,13,9,16,16,4,4,0


In [ ]:
resumen(X_nuevo)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SEXO,int64,2,27869,0,0.00%
TIPO_INGRESO,int64,2,27869,0,0.00%
CUIDADOS_INTENSIVOS,int64,2,27869,0,0.00%
DXR_1,int64,15,27869,0,0.00%
DXR_3,int64,15,27869,0,0.00%
SITUACION_ALTA,int64,2,27869,0,0.00%
TIPO_SERVICIO,int64,4,27869,0,0.00%
DX_EGRESO_AGRUPADO,int64,14,27869,0,0.00%
PROC_1_GRUPO,int64,17,27869,0,0.00%
PROC_2_GRUPO,int64,16,27869,0,0.00%



🧾 El DataFrame tiene un total de **14** columnas y **27869** filas.


##**Pruebas de Targets**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y['GRD_GRUPO_2DIG'], stratify=y['GRD_GRUPO_2DIG'], test_size=0.2, random_state=42)

In [ ]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [ ]:
def objective(trial):
    params = {
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'task_type': 'GPU',
        'random_seed': 42,
        'verbose': 0,

        'iterations': trial.suggest_int('iterations', 300, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.2, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 5.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 5.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
    }

    model = CatBoostClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=30,
        verbose=False
    )

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, timeout=600)  # ligero


# 4. Resultados
print("🏆 Mejor accuracy en validación:", study.best_value)
print("🧪 Mejores hiperparámetros encontrados:")
print(study.best_params)

# 5. Entrenar el mejor modelo
best_model = CatBoostClassifier(**study.best_params,
                                loss_function='MultiClass',
                                task_type='GPU',
                                eval_metric='MultiClass',
                                random_seed=42)

best_model.fit(X_train, y_train)

# 6. Evaluar
test_preds = best_model.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print(f"🎯 Accuracy en test: {test_acc:.4f}")

# 7. Métricas adicionales (opcional pero recomendado)
print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, test_preds))

print("🧩 Matriz de confusión:")
print(confusion_matrix(y_test, test_preds))

[I 2025-07-20 17:23:09,430] A new study created in memory with name: no-name-b482a0a7-6ca1-4e83-8326-91ae60d9e139
[I 2025-07-20 17:23:14,050] Trial 0 finished with value: 0.7077502691065662 and parameters: {'iterations': 720, 'depth': 5, 'learning_rate': 0.1527171353309801, 'l2_leaf_reg': 3.825042349173353, 'border_count': 34, 'random_strength': 0.032119995546488436, 'bagging_temperature': 0.7928842318711097, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.7077502691065662.
[I 2025-07-20 17:23:19,661] Trial 1 finished with value: 0.7093649085037675 and parameters: {'iterations': 441, 'depth': 7, 'learning_rate': 0.10631483678028752, 'l2_leaf_reg': 0.5043806703736601, 'border_count': 117, 'random_strength': 0.16535969537421488, 'bagging_temperature': 0.1686481684837402, 'grow_policy': 'Depthwise'}. Best is trial 1 with value: 0.7093649085037675.
[I 2025-07-20 17:23:23,613] Trial 2 finished with value: 0.6912450663796197 and parameters: {'iterations': 500, 'depth': 4, 'lea

🏆 Mejor accuracy en validación: 0.7134912091855041
🧪 Mejores hiperparámetros encontrados:
{'iterations': 951, 'depth': 5, 'learning_rate': 0.06116419603123537, 'l2_leaf_reg': 0.5302988518803281, 'border_count': 90, 'random_strength': 4.719648619468346, 'bagging_temperature': 0.8350974792444961, 'grow_policy': 'Depthwise'}
0:	learn: 2.7247265	total: 10.5ms	remaining: 9.93s
1:	learn: 2.4902148	total: 19ms	remaining: 9s
2:	learn: 2.3353945	total: 27.2ms	remaining: 8.61s
3:	learn: 2.2132977	total: 36.8ms	remaining: 8.71s
4:	learn: 2.1126572	total: 45.1ms	remaining: 8.54s
5:	learn: 2.0291962	total: 53.4ms	remaining: 8.41s
6:	learn: 1.9572313	total: 61.8ms	remaining: 8.33s
7:	learn: 1.8961677	total: 70ms	remaining: 8.26s
8:	learn: 1.8417949	total: 78.3ms	remaining: 8.19s
9:	learn: 1.7904417	total: 86.6ms	remaining: 8.15s
10:	learn: 1.7464841	total: 94.9ms	remaining: 8.11s
11:	learn: 1.7068799	total: 103ms	remaining: 8.07s
12:	learn: 1.6673145	total: 111ms	remaining: 8.03s
13:	learn: 1.633600

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y['GRD_AGRUPADO'], stratify=y['GRD_AGRUPADO'], test_size=0.2, random_state=42)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, timeout=600)  # ligero


# 4. Resultados
print("🏆 Mejor accuracy en validación:", study.best_value)
print("🧪 Mejores hiperparámetros encontrados:")
print(study.best_params)

# 5. Entrenar el mejor modelo
best_model2 = CatBoostClassifier(**study.best_params,
                                loss_function='MultiClass',
                                task_type='GPU',
                                eval_metric='MultiClass',
                                random_seed=42)

best_model2.fit(X_train, y_train)

# 6. Evaluar
test_preds = best_model2.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print(f"🎯 Accuracy en test: {test_acc:.4f}")

# 7. Métricas adicionales (opcional pero recomendado)
print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, test_preds))

print("🧩 Matriz de confusión:")
print(confusion_matrix(y_test, test_preds))

[I 2025-07-20 17:27:51,058] A new study created in memory with name: no-name-55bb1755-fda9-45b1-a837-f2b01bfce528
[I 2025-07-20 17:27:58,819] Trial 0 finished with value: 0.2360961607463222 and parameters: {'iterations': 361, 'depth': 8, 'learning_rate': 0.1941427014434997, 'l2_leaf_reg': 0.05967119532929509, 'border_count': 67, 'random_strength': 2.248281944801999, 'bagging_temperature': 0.017666927995451998, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.2360961607463222.
[I 2025-07-20 17:28:15,649] Trial 1 finished with value: 0.41083602439899536 and parameters: {'iterations': 843, 'depth': 4, 'learning_rate': 0.075345930468388, 'l2_leaf_reg': 0.010837435923617193, 'border_count': 44, 'random_strength': 0.5961970554739564, 'bagging_temperature': 0.39229271108245445, 'grow_policy': 'SymmetricTree'}. Best is trial 1 with value: 0.41083602439899536.
[I 2025-07-20 17:28:34,924] Trial 2 finished with value: 0.4203444564047363 and parameters: {'iterations': 376, 'depth': 5

🏆 Mejor accuracy en validación: 0.4223179045568712
🧪 Mejores hiperparámetros encontrados:
{'iterations': 802, 'depth': 6, 'learning_rate': 0.02171510913091446, 'l2_leaf_reg': 0.060114505427681854, 'border_count': 60, 'random_strength': 0.047549225965475096, 'bagging_temperature': 0.8482869697577089, 'grow_policy': 'Depthwise'}
0:	learn: 4.3191863	total: 74ms	remaining: 59.3s
1:	learn: 4.1437773	total: 152ms	remaining: 1m
2:	learn: 4.0136147	total: 224ms	remaining: 59.5s
3:	learn: 3.8981470	total: 298ms	remaining: 59.5s
4:	learn: 3.7969468	total: 374ms	remaining: 59.5s
5:	learn: 3.7076737	total: 444ms	remaining: 58.9s
6:	learn: 3.6321348	total: 516ms	remaining: 58.6s
7:	learn: 3.5657497	total: 587ms	remaining: 58.3s
8:	learn: 3.5046370	total: 660ms	remaining: 58.1s
9:	learn: 3.4480289	total: 734ms	remaining: 58.1s
10:	learn: 3.3958984	total: 809ms	remaining: 58.1s
11:	learn: 3.3474304	total: 900ms	remaining: 59.3s
12:	learn: 3.3029396	total: 972ms	remaining: 59s
13:	learn: 3.2572998	tot

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y['CODIGOS_SOLOS'], stratify=y['CODIGOS_SOLOS'], test_size=0.2, random_state=42)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, timeout=600)  # ligero


# 4. Resultados
print("🏆 Mejor accuracy en validación:", study.best_value)
print("🧪 Mejores hiperparámetros encontrados:")
print(study.best_params)

# 5. Entrenar el mejor modelo
best_model3 = CatBoostClassifier(**study.best_params,
                                loss_function='MultiClass',
                                task_type='GPU',
                                eval_metric='MultiClass',
                                random_seed=42)

best_model3.fit(X_train, y_train)

# 6. Evaluar
test_preds = best_model3.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print(f"🎯 Accuracy en test: {test_acc:.4f}")

# 7. Métricas adicionales (opcional pero recomendado)
print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, test_preds))

print("🧩 Matriz de confusión:")
print(confusion_matrix(y_test, test_preds))

[I 2025-07-20 17:42:28,393] A new study created in memory with name: no-name-8a9956ce-8093-4eda-816b-ef0830788c91
[I 2025-07-20 17:43:07,488] Trial 0 finished with value: 0.49354144241119485 and parameters: {'iterations': 574, 'depth': 6, 'learning_rate': 0.02173802683131779, 'l2_leaf_reg': 0.06714608838712277, 'border_count': 101, 'random_strength': 4.94114045951407, 'bagging_temperature': 0.9716389124604853, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 0.49354144241119485.
[I 2025-07-20 17:43:10,977] Trial 1 finished with value: 0.41478292070326517 and parameters: {'iterations': 669, 'depth': 7, 'learning_rate': 0.1502476744716883, 'l2_leaf_reg': 0.11957418267155694, 'border_count': 41, 'random_strength': 0.006071795444880775, 'bagging_temperature': 0.3743404741652617, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.49354144241119485.
[I 2025-07-20 17:44:13,507] Trial 2 finished with value: 0.43559382848941514 and parameters: {'iterations': 466, 'depth': 9,

🏆 Mejor accuracy en validación: 0.49354144241119485
🧪 Mejores hiperparámetros encontrados:
{'iterations': 574, 'depth': 6, 'learning_rate': 0.02173802683131779, 'l2_leaf_reg': 0.06714608838712277, 'border_count': 101, 'random_strength': 4.94114045951407, 'bagging_temperature': 0.9716389124604853, 'grow_policy': 'Depthwise'}
0:	learn: 4.1122204	total: 78.8ms	remaining: 45.1s
1:	learn: 3.8943503	total: 150ms	remaining: 42.9s
2:	learn: 3.7326871	total: 222ms	remaining: 42.2s
3:	learn: 3.6067171	total: 299ms	remaining: 42.6s
4:	learn: 3.5010222	total: 371ms	remaining: 42.2s
5:	learn: 3.4087450	total: 444ms	remaining: 42s
6:	learn: 3.3279382	total: 519ms	remaining: 42s
7:	learn: 3.2553456	total: 606ms	remaining: 42.9s
8:	learn: 3.1922502	total: 701ms	remaining: 44s
9:	learn: 3.1326212	total: 768ms	remaining: 43.3s
10:	learn: 3.0754661	total: 841ms	remaining: 43s
11:	learn: 3.0260936	total: 918ms	remaining: 43s
12:	learn: 2.9791924	total: 994ms	remaining: 42.9s
13:	learn: 2.9342240	total: 1.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
